# Imports and program variables


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, MaxPooling1D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import pandas as pd

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

from sklearn.model_selection import train_test_split # to split dataset into train and test
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from sklearn import preprocessing 

import sys
sys.path.append('../')
from time_series_dataset_loader import TimeSeriesDatasetLoader


Using TensorFlow backend.
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  

In [3]:
def discretization(X):
    new_dataset = []
    for data_instance in X:
        new_instance = []
        for row in data_instance:
            new_instance.append(np.average(row[2:]))
        new_dataset.append(new_instance)
        
    return np.asarray(new_dataset)

In [4]:
def linearize_dataset(dataset):
    ''' Returns a 1 dimensional representation from the dataset '''
    new_dataset = []

    for data_instance in dataset:
        new_dataset_cell = []
        for t_instance in data_instance:
            for value in t_instance[2:]:
                new_dataset_cell.append(value)

        new_dataset.append(new_dataset_cell)
    return np.asarray(new_dataset)
    
    

In [5]:
def preprocessing_data(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    
    max_len = len(X[0])
    for row in X:
        if len(row) > max_len:
            max_len = len(row)

    X = pad_sequences(X, maxlen=max_len, padding='post', dtype='float64')
    X = linearize_dataset(X)
    return X,y
    

In [6]:
def add_padding(X,y):
    X = np.asarray(X)
    y = np.asarray(y)
    
    max_len = len(X[0])
    for row in X:
        if len(row) > max_len:
            max_len = len(row)

    X = pad_sequences(X, maxlen=max_len, padding='post', dtype='float64')
    return X, y

In [40]:
d

In [8]:
DATASET_PATH = '../../datasets/Original/MFCC_5'

## Setting up train and test datasets

In [15]:
dataset_loader = TimeSeriesDatasetLoader(DATASET_PATH)
X, y = dataset_loader.get_dataset(type_='emotion_type')


In [74]:
# X, y = balance_dataset(X, y)

In [ ]:
X, y = preprocessing_data(X,y)


In [16]:
X = discretization(X)
X, y = add_padding(X,y)

In [37]:
#from pandas_profiling import ProfileReport
#df = pd.DataFrame(y)
#profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})

In [17]:
X[0][0]

1.6891301285714289

In [18]:
X = preprocessing.normalize(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)


In [80]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [46]:
model = get_model()

In [ ]:
X_traincnn = np.expand_dims(X_train, axis=0)
X_testcnn = np.expand_dims(X_test, axis=0)

In [28]:
X_train = X_train.reshape(1, len(X_train), 504)
X_test = X_test.reshape(1, len(X_test), 504)

In [82]:
embedding_size  = 128
kernel_size = 5
filters = 64
pool_size = 4

lstm_output_size = 70

# Training
batch_size = 64
epochs = 100

model = Sequential()
model.add(Embedding(1, embedding_size, input_length=504))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(3))
model.add(Activation('sigmoid'))


from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))



Train on 998 samples, validate on 428 samples
Epoch 1/100
998/998 [==============================] - 7s 7ms/step - loss: 1.1022 - accuracy: 0.2275 - val_loss: 1.0966 - val_accuracy: 0.4112
Epoch 2/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0964 - accuracy: 0.4248 - val_loss: 1.0918 - val_accuracy: 0.4112
Epoch 3/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0912 - accuracy: 0.4248 - val_loss: 1.0875 - val_accuracy: 0.4112
Epoch 4/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0864 - accuracy: 0.4248 - val_loss: 1.0836 - val_accuracy: 0.4112
Epoch 5/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0823 - accuracy: 0.4248 - val_loss: 1.0799 - val_accuracy: 0.4112
Epoch 6/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0784 - accuracy: 0.4248 - val_loss: 1.0765 - val_accuracy: 0.4112
Epoch 7/100
998/998 [==============================] - 7s 7ms/step - loss: 1.0748 - accuracy: 0.

In [ ]:
batch_size = 256
epochs = 500
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../model_checkpoints/two_split.h5', save_best_only=True, monitor='val_loss', mode='min')

In [49]:
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))
# , callbacks=[mcp_save, lr_reduce]

ValueError: Error when checking target: expected dense_5 to have shape (1,) but got array with shape (3,)

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 504, 128)          768       
_________________________________________________________________
activation_1 (Activation)    (None, 504, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 504, 64)           41024     
_________________________________________________________________
activation_2 (Activation)    (None, 504, 64)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 504, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 63, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 63, 32)           

array([[[  6.404657  ,  -1.402514  ,  -2.236425  , ...,   0.        ,
           0.        ,   0.        ],
        [  0.8584499 ,  -0.06234932,   0.6381655 , ...,   0.        ,
           0.        ,   0.        ],
        [  5.846846  , -10.07883   , -11.02912   , ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [ 12.99762   ,  -7.800581  ,  -1.155792  , ...,  -0.5558    ,
          -1.747546  ,   3.087212  ],
        [  3.280588  ,   2.007889  ,  -2.019495  , ...,  -0.9734643 ,
           0.8074539 ,   2.71006   ],
        [  5.084667  ,  -3.900648  ,  12.28177   , ...,  -0.7672844 ,
           2.181755  ,   4.326027  ]]])

In [ ]:
''' More Preprocessing '''

In [129]:
from sklearn.preprocessing import RobustScaler

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = RobustScaler()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 


In [16]:
from sklearn.preprocessing import Normalizer

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = Normalizer()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 

In [18]:
X[0][0]

array([ 0.        ,  0.        ,  0.28221284,  0.08020835, -0.09187018,
       -0.10788761,  0.62792463,  0.3675004 ,  0.16599244,  0.18636285,
       -0.35420188,  0.19850305, -0.14629544, -0.18601133, -0.21646694,
        0.18571278])

In [16]:
X_train.reshape(1)

ValueError: cannot reshape array of size 502992 into shape (1,)

In [ ]:
''' Finishing Preprocessing '''

In [ ]:
# See X and y details
print(X[:2])
print(X.shape)

print(y[:3])
print(y.shape)

In [41]:
# Split the dataset in train and test

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

print("\ny_train:\n")
print(y_train[:3])
print(y_train.shape)

print("\ny_test:\n")
print(y_test[:3])
print(y_test.shape)


X_train:

9.105775999999999
(1364, 504)

X_test:

-0.1714754
(585, 504)

y_train:

[2 1 2]
(1364,)

y_test:

[1 2 2]
(585,)


In [42]:
# Create categorical matrices



In [33]:
num_rows = X[0].shape[0]
num_columns = X[0].shape[1]
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

IndexError: tuple index out of range

## Building and Compiling Model

In [46]:
# top-k category accuracy
import functools
import keras
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])

In [47]:
# Define bath and epochs
batch_size = 256
epochs = 400
checkpoint_file = '../models/model_checkpoints/original_window_2.h5'
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../models/model_checkpoints/two_split.h5', save_best_only=True, monitor='val_loss', mode='min')
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), callbacks=[mcp_save, lr_reduce])

ValueError: Error when checking input: expected conv1d_3_input to have 3 dimensions, but got array with shape (1364, 504)

In [25]:
accuracy_list = cnnhistory.history['val_accuracy']
highest_index = cnnhistory.history['val_accuracy'].index(np.sort(cnnhistory.history['val_accuracy'])[-1])
print(cnnhistory.history['val_accuracy'][highest_index])

0.6239316463470459


In [ ]:
# without prepocessing: 0.5863248109817505
# robust preprocessing: 0.6034188270568848
# normalization: 0.6239316463470459

In [ ]:
import matplotlib.pyplot as plt

# Model Summary

model.summary()

# PLT History info
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()